In [1]:
import yfinance as yf
import pandas as pd
import numpy as np 

from AA import AssetDataDownloader, AssetAllocation

In [2]:
downloader = AssetDataDownloader()

assets = ["MSFT", "COST", "GOOG", "SYK", "FANG", "ODFL", "AAPL"] 
benchmark = '^GSPC'  
start_date = '2019-01-01'
end_date = '2023-12-31'
rf = .065

asset_prices, benchmark_prices = downloader.download_data(start_date= start_date, end_date= end_date,
                                                          assets= assets, benchmark=benchmark)

[*********************100%%**********************]  7 of 7 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
Asset_allocator = AssetAllocation(asset_prices = asset_prices,benchmark_prices = benchmark_prices, rf=rf)


In [4]:
Asset_allocator.run_optimizations()

C:\Users\lalos\anaconda3\lib\site-packages\scipy\optimize\optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


,Max Sharpe Ratio Weights (%),Max Omega Ratio Weights (%)
AAPL,30.160994,94.00000
COST,34.438765,1.00000
FANG,5.513560,1.00000
GOOG,1.000000,1.00000
MSFT,1.000000,1.00000
ODFL,26.886681,1.00000
SYK,1.000000,1.00000
Optimization Value,0.071535,1.15385


In [4]:
optimized_weights, sharpe = Asset_allocator.optimize_max_sharpe()
optimized_weights, sharpe 

(array([0.30160984, 0.3443876 , 0.05513553, 0.01      , 0.01      ,
        0.26886704, 0.01      ]),
 0.07153519541927252)

In [5]:
optimized_weights, omega = Asset_allocator.optimize_for_omega()
optimized_weights, omega 

(array([0.94, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]), 1.153849369107957)

In [4]:
optimized_weights, minvar = Asset_allocator.minimize_var()
optimized_weights, minvar

C:\Users\lalos\anaconda3\lib\site-packages\scipy\optimize\optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


(array([0.01      , 0.63598266, 0.05115126, 0.06457697, 0.01      ,
        0.02426662, 0.20402248]),
 0.022357498139065436)

In [5]:
optimized_weights, minvar = Asset_allocator.minimize_var()
optimized_weights, minvar

(array([0.01      , 0.63598322, 0.05115135, 0.06457652, 0.01      ,
        0.02426611, 0.2040228 ]),
 0.022357496307870397)

In [4]:
optimized_weights, minvar = Asset_allocator.minimize_var()
optimized_weights, minvar

C:\Users\lalos\anaconda3\lib\site-packages\scipy\optimize\optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


(array([0.01      , 0.63598308, 0.05115124, 0.06457685, 0.01      ,
        0.02426663, 0.2040222 ]),
 7.120703247877434)

In [8]:
asset_prices.iloc[-1].dot(optimized_weights) * 0.022357496307870397

11.479485097951109

In [9]:
asset_returns = asset_prices.pct_change().dropna()
benchmark_returns = benchmark_prices.pct_change().dropna()
asset_cov_matrix = asset_returns.cov()
rf = .1165

In [7]:
portfolio_rend

0.001204447545667817

In [6]:
asset_returns = asset_prices.pct_change().dropna()
asset_cov_matrix = asset_returns.cov()
rf = .1165

average_asset_returns = asset_prices.pct_change().dropna().mean()
start_weights = np.full(len(average_asset_returns), 1/len(average_asset_returns))
portfolio_rend = np.dot(average_asset_returns, start_weights)

variance = np.dot(np.dot(start_weights, asset_cov_matrix), start_weights)
volatility = np.sqrt(variance)

rf = rf/252

Sharpe = (portfolio_rend - rf)/volatility

In [4]:
import numpy as np
import scipy.optimize as sco

asset_returns = asset_prices.pct_change().dropna()
asset_cov_matrix = asset_returns.cov()
average_asset_returns = asset_returns.mean()
start_weights = np.full(len(average_asset_returns), 1 / len(average_asset_returns))

# Tasa libre de riesgo anual, ajustada a diario
rf_daily = rf / 252

# Función para calcular el negativo del ratio de Sharpe
def neg_sharpe_ratio(weights, average_asset_returns, cov_matrix, rf_daily):
    portfolio_return = np.dot(weights, average_asset_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - rf_daily) / portfolio_volatility
    return -sharpe_ratio  # Negativo porque estamos minimizando

# Restricciones y límites para los pesos
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # La suma de los pesos debe ser 1
bounds = tuple((0, 1) for _ in range(len(start_weights)))  # Los pesos deben estar entre 0 y 1

# Optimización
opts = sco.minimize(neg_sharpe_ratio, start_weights, args=(average_asset_returns, asset_cov_matrix, rf_daily),
                    method='SLSQP', bounds=bounds, constraints=constraints)

# Resultados de la optimización
optimized_weights = opts.x
optimized_sharpe = -opts.fun  # Negativo porque minimizamos el negativo del ratio de Sharpe

print("Pesos optimizados:", optimized_weights)
print("Ratio de Sharpe optimizado:", optimized_sharpe)


Pesos optimizados: [3.13442417e-01 3.56327240e-01 5.77706230e-02 4.44793941e-17
 0.00000000e+00 2.72459719e-01 3.52365706e-19]
Ratio de Sharpe optimizado: 0.07192814647143334


In [44]:
Sharpe

0.056991422047103946

In [19]:
from scipy.optimize import minimize

In [48]:
np.dot( start_weights,np.dot(start_weights, asset_cov_matrix))

0.0006458550742122128

In [38]:
volatility

0.02541367887993025

In [37]:
asset_cov_matrix

,AAPL,GOOG,MSFT
AAPL,0.000868,0.000537,0.000686
GOOG,0.000537,0.000585,0.000572
MSFT,0.000686,0.000572,0.000769


In [5]:
average_asset_returns

AAPL    0.002769
GOOG    0.001252
MSFT    0.001711
dtype: float64

In [14]:
portfolio_market_gap = asset_returns - benchmark_returns.values
downside = portfolio_market_gap[portfolio_market_gap < 0].fillna(0).std()
upside = portfolio_market_gap[portfolio_market_gap > 0].fillna(0).std()
omega = upside/downside
omega

AAPL    1.365584
GOOG    1.158482
MSFT    1.201645
dtype: float64

In [13]:
portfolio_market_gap

,AAPL,GOOG,MSFT
Date,,,
2020-01-03,-0.002662,0.002153,-0.005392
2020-01-06,0.004435,0.021124,-0.000949
2020-01-07,-0.001900,0.002179,-0.006314
2020-01-08,0.011184,0.002978,0.011026
2020-01-09,0.014586,0.004389,0.005838
...,...,...,...
2020-12-23,-0.007722,0.004406,-0.013785
2020-12-24,0.004175,0.000198,0.004291
2020-12-28,0.027043,0.012694,0.001199
